<a href="https://colab.research.google.com/github/DS-Jerry-in-Taiwan/project-set/blob/main/regression_clv_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Lifetime Value and Boston House Price

- <font size=4>[回歸實戰](https://www.xuemi.co/programs/8d6e9c07-87c7-4b34-99c7-2616d9bcc605/contents?back=project_50b678a6-14b0-4148-b706-724d7e834e20]</font>
- <font size=3>https://reurl.cc/8Gagoy</font>

# 載入程式庫及必要定義

In [ ]:
try:
    from google.colab import drive, files
    in_colab = True
except ModuleNotFoundError:
    in_colab = False

if in_colab:
    home_dir = ''
    drive.mount('/content/drive')
    groot_dir = '/content/drive/MyDrive/adventure_time/'
else:
    from pathlib import Path
    home_dir = str(Path.home())
    groot_dir = home_dir + '/Google Drive/adventure_time/'

import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=14)
mpl.rc('ytick', labelsize=14)
mpl.rc('font', size=14)

from datetime import datetime
from dateutil.relativedelta import *
import matplotlib.pyplot as plt
import sklearn
assert sklearn.__version__ >= "0.20"
import seaborn as sns
import pandas as pd
import numpy as np
import math
import os
import sys
import gdown
import requests
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
from pandas.plotting import register_matplotlib_converters

figure_dir = groot_dir + 'figure/regression/'
data_dir = groot_dir + 'regression/'

gfigure = lambda name: figure_dir + name + '.png'
output_fig = lambda name: plt.savefig( gfigure(name), dpi = 300)

local_time = lambda x, offset: x + relativedelta(hours= offset)
def local_now(hours = 8):
    return datetime.now() + relativedelta(hours = hours if in_colab else 0)

def print_now():
    return print(local_now())

def print_local_now():
    return print('Local Time:', local_now())

def DropboxLink(did, fname):
    return 'https://dl.dropboxusercontent.com/s/%s/%s' % \
    (did, fname)

def fetch_gdrive_file(fid, local_save):
    remote_url = 'https://drive.google.com/uc?id=' + fid
    gdown.download(remote_url, local_save, quiet = False)

def fetch_file_via_requests(url, save_in_dir):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    output_fpath = save_in_dir + local_filename
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(output_fpath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return output_fpath
        
def acct_string(num):
    s0 = str(num)
    if len(s0) <=3:
        return s0  
    num_section = int(len(s0)/3)
    remaining_start = len(s0) % 3
    s = s0[:remaining_start]
    for i in range(num_section):
        s += ',%s' % s0[remaining_start + i*3 :remaining_start + (i+1)*3]   
    return s

def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

def start_plot(figsize=(10, 8), style = 'whitegrid'):
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(1,1)
    plt.tight_layout()
    with sns.axes_style(style):
        ax = fig.add_subplot(gs[0,0])
    return ax

EX1DATA = '147xBeCECYur0FxDyly-oG2BqsqEH2Mxm'
EX1DATA2 = '101qw-9OkjCxwuSkBJUBaGURRWpZbFKOe'
EX5DATA = '1nNM8CN9CkRfjipRx1qJYZhSobmABVL1J'
ADVER = '1xFMcCuiMgX9VnelDtbyyV9rXBMFerx8k'
TAIWAN_CSV = '1I5yqulrZSHPSQkxT3oqt_3uVAhPolOEP'
JHU_CSSE = 'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/'
MNIST_TRAIN = '1E-uJ0zqqAfpsVjoOSzqF5TXhDfPNlkQ5'
MNIST_TRAIN_LABEL = '13clNJ2cd2I90W3DEkDBKjZSDNNEqqx3B'
MNIST_TEST = '1zVpVHJl5YABa3qExt1K-O3WaEHXTJekg'
MNIST_TEST_LABEL = '1qci_-dqubnRN-cdrCsbYaUAxyO7_jH9z'

print('\nRunning on %s' % sys.platform)
print('Python Version', sys.version)
print('Data storage points to ==>', data_dir)

print('\nThis module is amied to leran regression basics...') 
print('\nLibraries and dependenciess imported')
print_now()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

# 下載資料檔案

In [ ]:
fetch_file_via_requests(
    DropboxLink('e7lsf1k6258w7co', 'ec_201012_test_4.csv'), data_dir )

fetch_file_via_requests(
    DropboxLink('fwokyefy0looizp', 'ec_201012_train_4.csv'), data_dir )

In [ ]:
#檢視sklearn.metrics的score 可以有哪些選擇
import sklearn.metrics
for x in sklearn.metrics.SCORERS.keys():
    print(x)

# Preparing Data

訓練資料與測試（test）資料的名稱 ec_201012_train_4.csv 和 ec_201012_test_4.csv, 這兩個資料集的欄位相同，包括：

```
'CustomerID', 'date_size', 'date_recency', 'date_time_between',
'date_T', 'baseket_value_sum', 'baseket_value_mean',
'baseket_value_std', 'baseket_value_amax', 'baseket_value_amin',
'baseket_value_median', 'basket_size_sum', 'basket_size_mean',
'basket_size_std', 'basket_size_amax', 'basket_size_amin',
'basket_size_median', 'lag_12', 'lag_11', 'lag_10', 'lag_9', 'lag_8',
'lag_7', 'lag_6', 'lag_5', 'lag_4', 'lag_3', 'lag_2', 'lag_1', 'value'
```

最後一個欄位 <font color='brown'>‘value’</font> 是本案例需要預測的對象，是指未來一定期間內客戶購買的金額，也就是所謂的「貢獻值」。

### 資料命名規則

以下使用兩個dataset示範與練習
- CLV Case
    - train: 訓練集載入 DataFrame
    - test: 測試集載入 DataFrame
    - X_train, y_train (clv case 訓練集)
    - X_test, y_test (clv case 測試集)
- load_boston()
    - X_bos, y_bos (load_boston() 資料)
    - bos (資料打包為 Pandas DataFrame)

In [ ]:
#存一份資料副本，以便後續使用
train_csv = os.path.join(data_dir,"ec_201012_train_4.csv")
test_csv = os.path.join(data_dir,"ec_201012_test_4.csv")

In [ ]:
train = pd.read_csv(train_csv)
test = pd.read_csv(test_csv)

#製作train test data
x_train = train.drop(['CustomerID','value'],axis = 1)
y_train = train.value

x_test = test.drop(['CustomerID','value'],axis = 1)
y_test = test.value


In [ ]:
from sklearn.datasets import load_boston

data = load_boston() #載入boston房價資料
bos = pd.DataFrame(data = data['data'],
                   columns = data['feature_names'])

bos['y']= data['target']
x_bos = bos.drop(['y'],axis =1)
y_bos = bos.y

# from sklearn.datasets import load_boston

# data = load_boston()#載入boston房價資料
# bos = pd.DataFrame(data = data['data'], 
#     columns = data['feature_names'])
# bos['y'] = data['target']
# X_bos = bos.drop(['y'], axis = 1)
# y_bos = bos.y

#Boston dataset欄位定義

- CRIM per capita crime rate by town

- ZN proportion of residential land zoned for lots over 25,000 sq.ft.

- INDUS proportion of non-retail business acres per town

- CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)

- NOX nitric oxides concentration (parts per 10 million)

- RM average number of rooms per dwelling

- AGE proportion of owner-occupied units built prior to 1940

- DIS weighted distances to five Boston employment centres

- RAD index of accessibility to radial highways

- TAX full-value property-tax rate per $10,000

- PTRATIO pupil-teacher ratio by town

- B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town

- LSTAT % lower status of the population

- MEDV Median value of owner-occupied homes in $1000’s

## 常常需要載入的 Classes

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#是先定義好所需的函式，利後續重複使用
def simple_ols(xvec, yvec):
    Xadd = sm.add_constant(xvec)
    model = sm.OLS(yvec, Xadd).fit()
    return model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Modeling

In [ ]:
#檢視欄位(features)是否有缺失值
train.isnull().sum()
test.isnull().sum()

In [ ]:
#檢視欄位的關聯性，用以確認選取哪些欄位
cor = train.corr()
print(cor.value.sort_values(ascending=True))#直接他出value值的關聯性

#欄位太多造成熱圖失效
fig,ax = plt.subplots(figsize = (20,10))

sns.heatmap(cor,xticklabels = cor.columns,
            yticklabels = cor.columns, annot=True,
            cmap = 'RdBu', ax=ax)

## Customter Lifetime Value

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline

the_model = make_pipeline(
    #先將資料standardization
    #然後將standardiztion的資料丟入linearRegression
    StandardScaler(),
    LinearRegression()
)

#將global_var assign給 local_var
train_x = x_train
train_y = y_train

test_x = x_test
test_y = y_test

the_model.fit(train_x,train_y)
y_pred = the_model.predict(test_x)

#檢驗模型預的結果，使用指標：rmse(root-mean squred error)
rmse = np.sqrt(mean_squared_error(test_y,y_pred))
r2 = r2_score(test_y,y_pred)
r2,rmse#得出r^2:0.6182389801675541 rmse:206.16901350645512

In [ ]:
#To show what we got
#會員資料主要包四個面向：date(時間),basket_value(購物車金額),basket_size(購物車商品數),lag(過往的clv)
print(train_x.columns)#columns contain in x
print(train_x.iloc[:,:4])

date_data_set =train_x.filter(regex="date")
value_data_set =train_x.filter(regex="baseket_value")
size_data_set =train_x.filter(regex="basket_size")
lag_data_set =train_x.filter(regex="lag")

cor_date = date_data_set.corr()
cor_value = value_data_set.corr()
cor_size = size_data_set.corr()
cor_lag = lag_data_set.corr()


fig,ax = plt.subplots(2,2,figsize=(50,50))
sns.set(font_scale=2.5) 
sns.heatmap(cor_date,xticklabels = cor_date,
            yticklabels = cor_date, annot=True,
            cmap = 'copper_r', ax=ax[0][0])

sns.heatmap(cor_value,xticklabels = cor_value,
            yticklabels = cor_value, annot=True,
            cmap = 'copper_r', ax=ax[0][1])

sns.heatmap(cor_size,xticklabels = cor_size,
            yticklabels = cor_size, annot=True,
            cmap = 'copper_r', ax=ax[1][0])

sns.heatmap(cor_lag,xticklabels = cor_lag,
            yticklabels = cor_lag, annot=True,
            cmap = 'copper_r', ax=ax[1][1])




print(train_y.sort_values(ascending=False)[:10])#y is the clv of each customer



In [ ]:
#use scatter plot to visualize how data(y_pred,test_y) distribute
plt.figure(figsize=(20,10))
sns.regplot(y_pred,test_y)
plt.title("test_y and test_y regplot")

## load_boston

In [ ]:
#boston data develope
cor_bos = bos.corr()
cor_bos.y.sort_values(ascending=True)

fig,ax = plt.subplots(figsize = (20,10))
sns.heatmap(cor_bos,xticklabels=bos.columns,yticklabels=bos.columns,
            cmap="summer",annot= True,ax=ax)
#較不相關的變數屬於接近0的值

In [ ]:
train_x,test_x,train_y,test_y = \
      train_test_split(x_bos,y_bos,test_size = 0.2)


model = make_pipeline(
    MinMaxScaler(),
    LinearRegression()
)

model.fit(train_x, train_y)
y_pred = model.predict(test_x)
rmse = np.sqrt(mean_squared_error(test_y,y_pred))
r_2 = r2_score(test_y,y_pred)
print("way1",r_2,rmse)

#使用stats package 能較快數計算出統計指標
from statsmodels.tools.eval_measures import rmse
model = simple_ols(x_bos,y_bos)
rmse_score = rmse(y_bos, model.fittedvalues)

print("way2:",r_2, rmse_score)

# Charting

In [ ]:
def start_plot(figsize=(10, 8), style = 'whitegrid'):
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(1,1)
    plt.tight_layout()
    with sns.axes_style(style):
        ax = fig.add_subplot(gs[0,0])
    return ax

## 誤差值的統計分佈模型 Distribution of Residuals

- 功用：
- 分析面向：

- Points are **independent** of each other (residuals are uncorrelated)
- <font color='brown'>**residual ε are normally distributed with  μ = 0**</font>

In [ ]:
#了解residual的分佈，判斷預測值與真
from sklearn.linear_model import LinearRegression
#先訓練所需模型，在找出預測值
model = LinearRegression()
model.fit(train_x,train_y)
pred_y = model.predict(test_x)

#計算出預測值與實際值的差距就可得殘差項
ax=start_plot(figsize=(10,10))
sns.distplot(test_y-pred_y, bins = 20,color='orange',ax=ax)
ax.axvline(x= 0,ls = "--",c= 'brown')#殘差偏離x=0表示欄位間可能不互為獨立

In [ ]:
#製作殘差排序曲線圖（sorted-residual plot）
residual = (test_y - pred_y).sort_values(ascending=True)
ax = start_plot(figsize=(10,10))
ax.plot(range(len(residual)),residual)
ax.axhline(y=0,ls='--',c='red')

## 預測值的範圍與分佈

In [ ]:
print(pred_y)
print(test_y)

In [ ]:
#初步檢視預測值與實際值的分佈
model = LinearRegression()
model.fit(train_x,train_y)
pred_y = model.predict(test_x)
true_y = test_y
ax = start_plot(figsize=(10,10),style = 'darkgrid')
ax.plot(pred_y,label="Obsevation")
ax.plot(true_y,label="Prediction")
ax.legend(frameon=True,fontsize = 14,shadow=True)
#此圖並無法顯示分佈情形

In [ ]:
#方法一 ：seaborn繪製whiskerboxplot:運用dataframe方式同時繪製兩個箱形圖並進行比較分布差異
ndf = pd.DataFrame()
ndf['Obsevation'] = pred_y
ndf['Prediction'] = test_y
ax = start_plot(figsize=(10,10))
sns.boxplot(data=ndf,orient="h",ax=ax)

In [ ]:
#方法二 ：使用分佈圖比較兩者的分佈差異
ax=start_plot(figsize=(8,6))
sns.distplot(ndf.Obsevation)
sns.distplot(ndf.Prediction)

## $ y  - \hat y$ (預測效果) 的分析 （Regression Prediction Error）

- 功用：顯示預測值與實際值的散佈圖

- 分析面向：實際的模型預測與百分之百正確間的差異，可以顯示目前資料所製作的預測值是偏高或偏低，再進行調整

- [YellowBrick PredictionErrorPlot](https://www.scikit-yb.org/en/latest/api/regressor/peplot.html)

![texto alternativo](https://www.scikit-yb.org/en/latest/api/regressor/peplot-1.png)

In [ ]:
!pip3 install --upgrade yellowbrick

In [ ]:

model= LinearRegression()
model.fit(train_x,train_y)
ax=start_plot(figsize=(20,10),style="darkgrid")
pred_y = model.predict(test_x)
sns.regplot(test_y,pred_y)
# 調整座標軸，讓圖型成正方形且兩軸刻度一致
#作法：將兩軸的最大/最小值調整成一致(為避免資料點貼著軸線，故加減做微調)
lim1 = min(min(test_y),min(pred_y))-50
lim2 = max(max(test_y),max(pred_y))+50
lim = [lim1,lim2]
ax.set_xlim(lim)
ax.set_ylim(lim)
#強化資料點的輪廓
plt.scatter(test_y,pred_y,c='green',edgecolors="navy",label=r"$'r^2=%.4f$"%r2_score(test_y,pred_y))
#加上對角線虛線並顯示其意義
ax.plot(lim,lim,ls="--",linewidth=5,c="orange",label = r'$y = \haty, identity$')
#加上圖例
ax.legend(loc='best',frameon=True,shadow=True,fontsize=14)

## Residuals Analysis 錯誤值的分析

- [Residuals Plot](https://www.scikit-yb.org/en/latest/api/regressor/residuals.html)

### My Residuals Plot

In [ ]:
plt.style.available

In [ ]:
model = LinearRegression()
model.fit(train_x,train_y)
pred_train_y = model.predict(train_x)
pred_test_y = model.predict(test_x)

train_residual = train_y - pred_train_y
test_residual = test_y - pred_test_y

train_r2 = r2_score(train_y,pred_train_y)
test_r2 = r2_score(test_y,pred_test_y)

In [ ]:
plt.style.use("seaborn-notebook")
ax= start_plot(figsize=(20,10),style="darkgrid")

sns.regplot(train_y,train_residual)
sns.regplot(test_y,test_residual)
plt.scatter(train_y,train_residual,edgecolors="navy",label="Train Residual " + \
            r'$r^2:%.4f$' % train_r2)
plt.scatter(test_y,test_residual,edgecolors='black',label="Test Residual " + \
            r'$r^2:%.4f$' % test_r2)


ax.axhline(y= 0,ls="--",c="darkslategray")
ax.legend(loc = "best",frameon=True,fancybox=True,shadow=True,fontsize=16)

## 學習曲線

In [ ]:
from sklearn.model_selection import learning_curve

#
# Simplified version of plot_learning_curves presented in 
#   https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
#

def my_plot_leaning_curves(model, X, y, cv=None, scoring = None,
        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 10)):
    
       
    fig,ax = plt.subplots(figsize = (10, 8))
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(model, X, y, cv=cv, n_jobs=n_jobs, 
            scoring = scoring,
            train_sizes=train_sizes,
            return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve


    ax.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    ax.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    ax.grid(b = 'on', ls = '--', alpha = 0.8)
    ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    ax.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")

    ax.legend(loc="best", fontsize = 14, shadow =True, frameon = True)

In [ ]:
my_plot_leaning_curves(LinearRegression(), X_bos, y_bos,
    train_sizes=np.linspace(.1, 1.0, 5),
    scoring='neg_root_mean_squared_error')

# Metrics & Outliers

## Cook's Distance
1.用以評估每個資料點對於mse的影響度

2.cook's distance大，表示資料點可能為離群值，故今其排除可提升模型的解釋能力

3.一般以整體cook's distance的三倍為界線，大於三倍則是為cook's distance大

- [Wikipedia](https://en.wikipedia.org/wiki/Cook%27s_distance)
- [Mathworks](https://www.mathworks.com/help/stats/cooks-distance.html#:~:text=Cook's%20distance%20is%20the%20scaled,on%20the%20fitted%20response%20values.)
- [Stackoverflow QA](https://stackoverflow.com/questions/51390196/how-to-calculate-cooks-distance-dffits-using-python-statsmodel)

- [MAPE formula](https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn/294069#294069)

- [Issue 15007](https://github.com/scikit-learn/scikit-learn/pull/15007)

>Simply said, Cook’s D is calculated by removing the ith data point from the model and recalculating the regression. All the values in the regression model are then observed whether changes have been detected after the removal of the point. This is an iterative way of examining the influence of that observation.



以 loas_boston 為例：

```
model = simple_ols(X_bos, y_bos)
infl = model.get_influence()
rdf = infl.summary_frame()
cooks_d = rdf['cooks_d']
```

#視覺化Cook's distance

In [ ]:

model = simple_ols(x_bos,y_bos)
inf1 = model.get_influence()#statmodle package現成的function
rdf = inf1.summary_frame()
cooks_d = rdf['cooks_d']
ax = start_plot(figsize=(20,10))
ax.vlines(range(len(rdf)),cooks_d,0,color='navy',label="Cook's distance")
ax.legend(frameon = True, fontsize = 14, shadow = True)


In [ ]:
cooks_d = rdf['cooks_d'].sort_values(ascending = True)
# type(cooks_d), cooks_d

#抓出top8 cook's distance value
top8 = list(cooks_d.index[:int(len(cooks_d)*0.08)])

#抓出大於cook's distance 平均值三倍的value
mu3 = 3*cooks_d.mean()
above3 = list(cooks_d.loc[cooks_d > mu3].index)

#製作兩個資料集：drop掉mu3與drop掉top8
bos2 = bos.drop(above3,axis = 0)
bos3 = bos.drop(top8,axis = 0)

In [ ]:
#比較排除前與排除後cross_val_score差異
from sklearn.model_selection import cross_val_score
scores = cross_val_score(LinearRegression(), x_bos,y_bos)
scores2 = cross_val_score(LinearRegression(),bos2.drop(['y'],axis =1),bos2.y)
scores3 = cross_val_score(LinearRegression(),bos3.drop(['y'],axis =1),bos3.y)


print("score of bos: ",scores,";",scores.mean(),";",scores.std())
print("score of bos2: ",scores2,";",scores2.mean(),";",scores2.std())
print("score of bos3: ",scores3,";",scores3.mean(),";",scores3.std())

- [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
- [What Is R Squared And Negative R Squared](http://www.fairlynerdy.com/what-is-r-squared/)
- [Can the multiple linear correlation coefficient be negative?](https://stats.stackexchange.com/questions/6181/can-the-multiple-linear-correlation-coefficient-be-negative)

### yellowbrick 版本

```
from yellowbrick.regressor import CooksDistance
from sklearn.datasets import load_boston

data = load_boston()
X, y = data['data'], data['target']

# Instantiate and fit the visualizer
visualizer = CooksDistance()
visualizer.fit(X, y)
visualizer.show()
```

## VIF(變異數膨脹因子)

>Variance Inflation Factor (VIF) is used to detect the presence of multicollinearity. Variance inflation factors (VIF) measure how much the variance of the estimated regression coefficients are inflated as compared to when the predictor variables are not linearly related.

>- 功用：計算兩兩欄位間的關聯性 --> 防止共線性發生

- [select_dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html)
- [endog, exog, what’s that?](https://www.statsmodels.org/stable/endog_exog.html)
- [variance_inflation_factor](https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
df=bos.select_dtypes(include=[np.number])
bos.select_dtypes(include=[np.number]).shape
range(df.shape[1])

[variance_inflation_factor(df.values,1)]

In [ ]:
import pandas as pd
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.datasets import load_boston

def vif_scores(df):
    df.dropna()
    # df = df._get_numeric_data()
    df = df.select_dtypes(include=[np.number])#篩選出數執行資料
    vif = pd.DataFrame()#另外創一個dataframe

    vif["VIF Factor"] = \
        [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]

    vif["features"] = df.columns
    return vif

def my_mape(estimator, X, y): 
    estimator.fit(X, y)
    y_pred = estimator.predict(X)
    y_true = y
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    

In [ ]:
bos2 = bos.copy()
vif = vif_scores(bos2.drop(['y'],axis=1))
vif_sort_idx = list(vif['VIF Factor'].sort_values(ascending=False).index)

#拿掉前三大VIF的變數
bos2 = bos2.drop(vif.iloc[vif_sort_idx[:3]]['features'],axis=1)
# bos2.info()

In [ ]:
#比較排除前與排除後cross_val_score差異
from sklearn.model_selection import cross_val_score
scores = cross_val_score(LinearRegression(), x_bos,y_bos)
scores2 = cross_val_score(LinearRegression(),bos2.drop(['y'],axis =1),bos2.y)



print("score of bos: ",scores,";",scores.mean(),";",scores.std())
print("score of bos2: ",scores2,";",scores2.mean(),";",scores2.std())


In [ ]:
 # Finalize and render the figure
from yellowbrick.regressor import ResidualsPlot

model = LinearRegression()
visualizer = ResidualsPlot(model)

visualizer.fit(train_x, train_y)  # Fit the training data to the visualizer
visualizer.score(test_x, test_y)  # Evaluate the model on the test data
visualizer             

# 降維

## Principal Component Analysis

- 共用：用數學工具(線性代數)在資料統計特性不影響下，減少資料維度(欄位)


- [線代啟示錄：主成分分析](https://ccjou.wordpress.com/2013/04/15/主成分分析/)
- [機器/統計學習:主成分分析(Principal Component Analysis, PCA)](https://medium.com/@chih.sheng.huang821/機器-統計學習-主成分分析-principle-component-analysis-pca-58229cd26e71)
- [主成分分析的原理](http://web.ntpu.edu.tw/~ccw/statmath/M_pca.pdf)
- [如何通俗易懂地讲解什么是 PCA 主成分分析？](https://www.zhihu.com/question/41120789)
- [scikit-learn PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
- [scikit-learn User Guide: 2.5. Decomposing signals in components (matrix factorization problems)](https://scikit-learn.org/stable/modules/decomposition.html#pca)

- [Principal Component Analysis (PCA) in Python](https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python)
- [Feature Selection Techniques in Machine Learning with Python](https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# model = LinearRegression()
#首先執行上述流程：建模、訓練、測試

model = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

model.fit(train_x,train_y)

# #接著計算模型預測的評估分數：RMSE、R2
pred_train_y = model.predict(train_x)
pred_test_y = model.predict(test_x)

train_rmse =  -np.sqrt(mean_squared_error(train_y,pred_train_y))
test_rmse =  -np.sqrt(mean_squared_error(test_y,pred_train_y))

train_r2 = model.score(train_x,train_y)
test_r2 = model.score(test_x,test_y)

print("training: R2 = %.4f; rmse = %.4f" % (train_r2,train_rmse)
)
print("testing: R2 = %.4f; rmse = %.4f" % (test_r2,test_rmse))

# model = make_pipeline(
#     StandardScaler(),
#     LinearRegression()
# )

# model.fit(X_train, y_train)

# yhat_train = model.predict(X_train)
# yhat_test = model.predict(X_test)


# TRAINING_RMSE = -np.sqrt(mean_squared_error(y_train, yhat_train))
# TEST_RMSE = -np.sqrt(mean_squared_error(y_test, yhat_test))

# TRAINING_SCORE = model.score(X_train, y_train)
# TEST_SCORE = model.score(X_test, y_test)

# print('training: score = %.4f, neg rmse = %.4f' % 
#       (TRAINING_SCORE, TRAINING_RMSE))
# # yhat = lin.predict(X_test)
# print('test: score = %.4f, neg rmse = %.4f' % 
#       (TEST_SCORE, TEST_RMSE))


In [ ]:
import logging
logging.getLogger("imported_module").setLevel(logging.INFO)
#預設資料欄位數
N = len(train_x.columns)

#create 一個等長、0組成的np.array
train_score = np.zeros(N)
test_score = np.zeros(N)

train_rmse = np.zeros(N)
test_rmse = np.zeros(N)

#make_pipepline串接資料格式：依照n(由多至少)的對資料進行降維，隨後丟入model進行訓練與測試
for i in np.arange(N,2,-1):
  model = make_pipeline(
      StandardScaler(),
      #針對資料做降維處理(降成n維)
      PCA(n_components=i),
      LinearRegression()
  )

  model.fit(train_x,train_y)
  #在降維後訓練並得到score，以做比較各維度的模型分數：R2、RMSE
  train_score[i-1] = model.score(train_x,train_y)
  test_score[i-1] = model.score(test_x,test_y)

  pred_train_y = model.predict(train_x)
  pred_test_y = model.predict(test_x)

  train_rmse[i-1] = -np.sqrt(mean_squared_error(train_y,pred_train_y))
  test_rmse[i-1] = -np.sqrt(mean_squared_error(test_y,pred_test_y))


#根據圖形結果分析：在test_set R2趨勢中，n=25,26達到相對高點;RMSE同時達相對低點
#以n=25,26的維度進行standardization後的linearregression model可以達到較優的表現
x_domain = np.arange(2,N,1)

fig,ax=plt.subplots(2,1,figsize=(20,10))

train_max=sum(train_score[2:]!=max(train_score[2:])) 
test_max=sum(test_score[2:]!=max(train_score[2:])) 
ax[0].plot(x_domain,train_score[2:],color="darkorange",label="Training Score")
ax[0].plot(x_domain,test_score[2:],color="darkred",label="Testing Score")

ax[0].vlines(test_max,ymin=0,ymax=max(test_score[2:]),linestyles='--',color='grey')
ax[0].vlines(train_max,ymin=0,ymax=max(train_score[2:]),linestyles='--',color='grey')

ax[0].legend(loc="best",frameon = True,fontsize=14,shadow=True)
ax[0].set_title("Score Trending",size=20)
ax[0].set_xticks(x_domain);#tips:
              #use a semi-colon after the plot command
              #will display the graph and stop the verbose
              #also,plt.show() supress unnecessary variable
              #https://stackoverflow.com/questions/12056115/disable-the-output-of-matplotlib-pyplot


# train_max=sum(train_rmse[2:]!=max(train_rmse[2:])) 
# test_max=sum(test_rmse[2:]!=max(train_rmse[2:])) 

ax[1].plot(x_domain,train_rmse[2:],color="darkgreen",label="Training RMSE")
ax[1].plot(x_domain,test_rmse[2:],color="darkblue",label="Testing RMSE")

# ax[1].vlines(train_max,ymin=max(train_rmse[2:]),ymax=0,linestyles='--',color='grey')
# ax[1].vlines(test_max,ymin=max(test_rmse[2:]),ymax=0,linestyles='--',color='grey')

ax[1].legend(loc="best",frameon = True,fontsize=14,shadow=True)
ax[1].set_title("RMSE Trending",size=20)
ax[1].set_xticks(x_domain);




### Give it a try

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline






def PCA_analysis(estimater,preprocessing, n_features, X_train, y_train, 
    X_test = None, y_test = None):

    N = n_features

    scores_train = np.zeros(N)
    scores_test = np.zeros(N)

    for i in np.arange(N, 2, -1):
        model = make_pipeline(
            preprocessing,
            PCA(n_components = i),
            estimater
        )
        model.fit(X_train, y_train)
        scores_train[i-1] = model.score(X_train, y_train)
        if X_test is not None:
            scores_test[i-1] = model.score(X_test, y_test)

    xdomain = np.arange(2, N, 1)
    ax = start_plot(figsize=(10,7))
    ax.plot(xdomain, scores_train[2:], label = 'Training Scores')
    if X_test is not None:
        ax.plot(xdomain, scores_test[2:], color = 'brown',
            label = 'Test Scores')
    ax.legend(loc='lower center', frameon=True,shadow=True,fancybox=True,fontsize=14)


In [ ]:
train_x.shape[1]

In [ ]:
PCA_analysis(LinearRegression(),MinMaxScaler(),train_x.shape[1],train_x,train_y,test_x,test_y)